In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup     
import pandas as pd   
import openpyxl
import time
import sys
import re
import math
import random

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

area_list = []     # 지역 리스트
title2 = [] # column 사업명
target2 = []       # column 대상
area2 = []         # column 지자체
url_all_list=[]    # URL
summary2 = []      # columne 기타

# 사용자로부터 입력 받기
for i in range(3):
    query_txt = input('지역명: ')
    area_list.append(query_txt)
    
cnt = 20
page_cnt = math.ceil(cnt / 10)

f_dir = input('저장 경로 (예:c:\\py_temp\\): ')
if f_dir =='' :
        f_dir = "c:\\py_temp\\"
fc_name = f_dir + '지역 정책.csv'
fx_name = f_dir + '지역 정책.xls'
print('=' *100)
    
# 사용자로부터 입력 받기
for i in area_list:
    print(i + '의 정책 20개를 모읍니다.')
    # 웹사이트 접속 + 검색
    url = 'https://www.youthcenter.go.kr/main.do'
    driver.get(url)
    try:
        driver.maximize_window()
    except:
        time.sleep(0.5)
    time.sleep(2)
    s_time = time.time( )
    element = driver.find_element(By.ID,'searchWordInput')
    driver.find_element(By.ID,'searchWordInput').click( )
    element.send_keys(i)
    element.send_keys("\n")
    time.sleep(2)
    driver.find_element(By.XPATH,'//*[@id="search_gnb"]/li[3]/a').click()
    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    search_con_list = soup.find('div','search_con_list').find_all('li')
    
    no = 1
    
    for i in search_con_list:
        title = str(i.find('a').find('strong').get_text())
        title = re.sub('(\d{4})년', '', title, 0).strip() # 정규표현식으로 년도 표기 제외
        print(no, '\n')
        print('사업명:', title)
        title2.append(title)
            
        summary = i.find('a').find('p').get_text()
        print('내용:', summary)
        summary2.append(summary)
            
        sub = i.find('a').find('div', 'search_list_info')
        area = str(sub.find_all('p')[0].get_text())
        area = re.sub('지역', '', area, 0).strip()
        print('대상:', area)
        area2.append(area)
            
        target = str(sub.find_all('p')[3].get_text())
        target = re.sub('대상연령', '', target, 0).strip()
        print('대상연령:', target)
        target2.append(target)
            
        url_1 = str(i.find('a')['onclick'])
        url_1 = re.sub('go_detail\(\'', '', url_1, 0).strip()
        url_1 = re.sub('\'\);', '', url_1, 0).strip()
        url_1 = re.sub('return false;', '', url_1, 0).strip()
            
        full_url = 'https://www.youthcenter.go.kr' + url_1
        time.sleep(1)
        driver.get(full_url)
        print('URL 주소:' , full_url)
        url_all_list.append(full_url)
            
        time.sleep(2)
            
        print('\n')
        no += 1
        if no > cnt + 1:
            break
            
# DataFrame 화
policy = pd.DataFrame()
policy['사업명'] = title2
policy['대상'] = target2
policy['기타'] = summary2
policy['URL'] = url_all_list

# csv 형태로 저장
policy.to_csv(fc_name, encoding = 'utf-8-sig', index = False)
print(" csv 파일 저장 경로: %s" %fc_name)

# 엑셀 형태로 저장
policy.to_excel(fx_name , index=False , encoding="UTF-8" , engine='openpyxl')
print(" xls 파일 저장 경로: %s" %fx_name) 

지역명: 원주
지역명: 경산
지역명: 김천
저장 경로 (예:c:\py_temp\): 
원주의 정책 20개를 모읍니다.
1 

사업명: 대학생 관광서포터즈 운영
내용: 원주 관광 홍보에 관심이 많은 대학생 서포터즈를 모집 및 활동 지원함으로써 관내 주요 관광지 및 축제 등의 SNS 홍보 활동 수행, 젊은 감각의 새로운 관광트렌드를 정책 반영
대상: 강원 원주시
대상연령: 만 19세 이상
URL 주소: https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?bizId=R2021031101401


2 

사업명: 원주 청년리더 아카데미
내용: 원주의 미래를 이끌어 갈 창조적인 청년 리더 발굴 및 역량 강화
대상: 강원 원주시
대상연령: 만 18세 ~ 39세
URL 주소: https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?bizId=R2021031101402


3 

사업명: 대학생 관광서포터즈 운영
내용: 원주 관광 홍보에 관심이 많은 대학생 서포터즈를 모집 및 활동 지원함으로써 관내 주요 관광지 및 축제 등의 SNS 홍보 활동 수행, 젊은 감각의 새로운 관광트렌드를 정책 반영
대상: 강원 원주시
대상연령: 만 19세 이상
URL 주소: https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?bizId=R2022031900076


4 

사업명: 거주지 이동 대학생 학자금 지원 사업
내용: 원주시 전입 관내 대학생의 최소한의 학자금(생활비) 1인당 10만원 지급(학기)
대상: 강원 원주시
대상연령: 제한없음
URL 주소: https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?bizId=R2021031101400


5 

사업명: 찾아가는 청년 희망 드림버스 운영
내용: 취업을 희망하는 청년들이 구인수요가 있는 지역의 우